In [8]:
# coding: utf-8
"""
Beating the benchmark @ Kaggle Springleaf
@author: Abhishek Thakur
"""
import sys
sys.path.append( "~/Users/wendellkuling/ipython/xgboost/wrapper" )

In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing, linear_model

XGBoostLibraryNotFound: Cannot find XGBoost Libarary in the candicate path, did you run build.sh in root path?
List of candidates:
/Users/wendellkuling/.local/lib/python2.7/site-packages/xgboost-0.4-py2.7.egg/xgboost/libxgboostwrapper.so
/Users/wendellkuling/.local/lib/python2.7/site-packages/xgboost-0.4-py2.7.egg/xgboost/../../wrapper/libxgboostwrapper.so
/Users/wendellkuling/.local/lib/python2.7/site-packages/xgboost-0.4-py2.7.egg/xgboost/./wrapper/libxgboostwrapper.so

In [ ]:
train = pd.read_csv('~/ipython/Kaggle_springleaf/Input/train.csv')
test = pd.read_csv('~/ipython/Kaggle_springleaf/Input/test.csv')

In [3]:
y = train.target.values
train = train.drop(['ID', 'target'], axis=1)
test = test.drop('ID', axis=1)

In [4]:
train = train.dropna(axis=1, thresh=2000)
test = test.dropna(axis=1, thresh=2000)

train = train.fillna(-1)
test = test.fillna(-1)

In [5]:
for f in train.columns:
    if train[f].dtype=='object': 
        print f
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))                    

VAR_0001
VAR_0005
VAR_0008
VAR_0009
VAR_0010
VAR_0011
VAR_0012
VAR_0043
VAR_0044
VAR_0073
VAR_0075
VAR_0156
VAR_0158
VAR_0159
VAR_0166
VAR_0167
VAR_0168
VAR_0169
VAR_0176
VAR_0177
VAR_0178
VAR_0179
VAR_0196
VAR_0200
VAR_0202
VAR_0204
VAR_0216
VAR_0217
VAR_0222
VAR_0226
VAR_0229
VAR_0230
VAR_0232
VAR_0236
VAR_0237
VAR_0239
VAR_0274
VAR_0283
VAR_0305
VAR_0325
VAR_0342
VAR_0352
VAR_0353
VAR_0354
VAR_0404
VAR_0466
VAR_0467
VAR_0493
VAR_1934


In [6]:
X = np.array(train)
X_test = np.array(test)
clf = xgb.XGBClassifier(n_estimators=50, nthread=-1, max_depth=17,
                        learning_rate=0.01, silent=False, subsample=0.8, colsample_bytree=0.7)

In [ ]:
clf.fit(X, y)

In [ ]:
preds = clf.predict_proba(X_test)[:,1]
sample = pd.read_csv('~/ipython/Kaggle_springleaf/Input/sample_submission.csv')
sample.target = preds
sample.to_csv('~/ipython/Kaggle_springleaf/Output/benchmarkxgb.csv', index=False)